In [ ]:
import pandas as pd
import numpy as np

# Cargar el archivo de Excel
file_path = "Tablas CSO.xlsx"  # Asegurate de tenerlo en el mismo directorio
xls = pd.ExcelFile(file_path)
df_hombres = pd.read_excel(xls, sheet_name='hombres')
df_mujeres = pd.read_excel(xls, sheet_name='mujeres')

# Parámetros
edades_interes = [20, 30, 40, 50, 60, 70]
anios = [1980, 2001, 2017]
i = 0.04  # tasa técnica
v = 1 / (1 + i)
capital = 10_000_000

# Función para calcular renta vitalicia inmediata e ilimitada
def calcular_renta_vitalicia(tabla, edades_interes, anios, capital):
    resultados = {"Edad": edades_interes}
    for anio in anios:
        primas = []
        for edad_inicial in edades_interes:
            qx_edad = tabla.loc[tabla['edad'] >= edad_inicial, anio].dropna().values
            if len(qx_edad) < 2:
                primas.append(np.nan)
                continue
            lx = [1.0]
            for q in qx_edad[:-1]:
                lx.append(lx[-1] * (1 - q))
            lx = np.array(lx)
            Lx = (lx[:-1] + lx[1:]) / 2
            v_pot = v ** np.arange(len(Lx))
            a_x = np.sum(Lx * v_pot)
            prima_pura = round(a_x * capital, 2)
            primas.append(prima_pura)
        resultados[anio] = primas
    return pd.DataFrame(resultados)

# Calcular primas para hombres y mujeres
df_primas_hombres = calcular_renta_vitalicia(df_hombres, edades_interes, anios, capital)
df_primas_mujeres = calcular_renta_vitalicia(df_mujeres, edades_interes, anios, capital)
